In [1]:
from dotenv import load_dotenv
from IPython.display import Markdown

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq
models = {"Google": ["gemma2-9b-it", "gemma-7b-it"],
          "Groq": ["llama3-groq-70b-8192-tool-use-preview", "llama3-groq-8b-8192-tool-use-preview"],
          "Meta": ["llama-3.1-70b-versatile", "llama-3.1-8b-instant", "llama-3.2-1b-preview", "llama-3.2-3b-preview", "llama-3.2-11b-vision-preview", "llama-3.2-90b-vision-preview", "llama-guard-3-8b", "llama3-70b-8192", "llama3-8b-8192"],
          "Mistral": ["mixtral-8x7b-32768"],
          "OpenAI": ["whisper-large-v3", "whisper-large-v3-turbo"]}
llm_model_groq = ChatGroq(temperature=0, model=models["Meta"][0])

In [3]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
urls = [
    "https://www.anthropic.com/research/many-shot-jailbreaking",
    "https://keivanjamali.com"
]
loader = WebBaseLoader(web_path=urls[0],
                     bs_kwargs=dict(
                         parse_only=bs4.SoupStrainer(
                             class_=("ReadingDetail_detail__wf2_W")
                         )
                     ))
docs = [loader.load()]
loader = WebBaseLoader(web_path=urls[1])
docs.append(loader.load())

docs_list = [item for sublist in docs for item in sublist]
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[[Document(metadata={'source': 'https://www.anthropic.com/research/many-shot-jailbreaking'}, page_content='We investigated a “jailbreaking” technique — a method that can be used to evade the safety guardrails put in place by the developers of large language models (LLMs). The technique, which we call “many-shot jailbreaking”, is effective on Anthropic’s own models, as well as those produced by other AI companies. We briefed other AI developers about this vulnerability in advance, and have implemented mitigations on our systems.The technique takes advantage of a feature of LLMs that has grown dramatically in the last year: the context window. At the start of 2023, the context window—the amount of information that an LLM can process as its input—was around the size of a long essay (~4,000 tokens). Some models now have context windows that are hundreds of times larger — the size of several long novels (1,000,000 tokens or more).The ability to input increasingly-large amounts of informatio

In [4]:
docs_list

[Document(metadata={'source': 'https://www.anthropic.com/research/many-shot-jailbreaking'}, page_content='We investigated a “jailbreaking” technique — a method that can be used to evade the safety guardrails put in place by the developers of large language models (LLMs). The technique, which we call “many-shot jailbreaking”, is effective on Anthropic’s own models, as well as those produced by other AI companies. We briefed other AI developers about this vulnerability in advance, and have implemented mitigations on our systems.The technique takes advantage of a feature of LLMs that has grown dramatically in the last year: the context window. At the start of 2023, the context window—the amount of information that an LLM can process as its input—was around the size of a long essay (~4,000 tokens). Some models now have context windows that are hundreds of times larger — the size of several long novels (1,000,000 tokens or more).The ability to input increasingly-large amounts of information

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300,
                                                                     chunk_overlap=50)
docs_splitted_1 = text_splitter.split_documents([docs_list[0]])    
docs_splitted_2 = text_splitter.split_documents([docs_list[1]])    

In [6]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Create vector store for the first URL (Anthropic research)
vector_sore_anthropic = Chroma.from_documents(
    documents=docs_splitted_1,  # assuming docs_splitted[0] holds the Anthropic data
    collection_name="rag-chroma-anthropic",
    embedding=HuggingFaceEmbeddings()
)
retriever_anthropic = vector_sore_anthropic.as_retriever()
print(vector_sore_anthropic._collection.count())

c:\Python Interpreters\in_LLM\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


8


In [7]:
# Create vector store for the second URL (Keivan's portfolio)
vector_sore_keivan = Chroma.from_documents(
    documents=docs_splitted_2,  # assuming docs_splitted[1] holds Keivan's portfolio data
    collection_name="rag-chroma-keivan",
    embedding=HuggingFaceEmbeddings()
)
retriever_keivan = vector_sore_keivan.as_retriever()
print(vector_sore_keivan._collection.count())

8


In [8]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

system_template = """You are a evaluator determining the relevance of a retrieved {document} to a user's query {question}. If the document contains keyword(s) or semantic meaning related to the question, mark it as relevant. Assign a binary score of 'yes' or 'no' to indicate the document's relevance to the question."""

system_prompt = SystemMessagePromptTemplate.from_template(
    input_variables=["documents", "question"],
    template=system_template)
human_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["question"],
    template="{question}"
)

grader_prompt = ChatPromptTemplate.from_messages(
    [system_prompt, human_prompt]
)
grader_prompt

ChatPromptTemplate(input_variables=['document', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['document', 'question'], input_types={}, partial_variables={}, template="You are a evaluator determining the relevance of a retrieved {document} to a user's query {question}. If the document contains keyword(s) or semantic meaning related to the question, mark it as relevant. Assign a binary score of 'yes' or 'no' to indicate the document's relevance to the question."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [9]:
prompt_template = """Given a user input {question}, your task is re-write or rephrase the question to optimize the query in order to imprive the content generation.
Your answer should only containt the re-writed question. Nothing else."""
system_prompt_2 = SystemMessagePromptTemplate.from_template(
    input_variables=["question"],
    template=prompt_template)
human_prompt_2 = HumanMessagePromptTemplate.from_template(
    input_variables=["question"],
    template="{question}"
)
re_write_prompt = ChatPromptTemplate.from_messages(
    [system_prompt_2, human_prompt_2]
)
re_write_prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Given a user input {question}, your task is re-write or rephrase the question to optimize the query in order to imprive the content generation.\nYour answer should only containt the re-writed question. Nothing else.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [10]:
from pydantic import BaseModel, Field

class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")

def get_score(doc) -> str:
    """Return the binary score as a stings."""
    return doc.binary_score


In [11]:
structured_llm_grader_groq = llm_model_groq.with_structured_output(GradeDocuments)

In [12]:
from langchain_community.tools import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)


In [13]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

c:\Python Interpreters\in_LLM\Lib\site-packages\langsmith\client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [21]:
def assess_retrieved_docs(query):
    """Rewrite and asses the relevanceof documents to a give query."""
    retrieval_grader = (grader_prompt
                        | structured_llm_grader_groq
                        | get_score)
    docs_anthropic = retriever_anthropic.get_relevant_documents(query)
    docs_keivan = retriever_keivan.get_relevant_documents(query)

    relevance_scores = {}
    relevance_docs = {}

    for idx, doc in enumerate(docs_anthropic):
        doc_txt = doc.page_content
        binary_score = retrieval_grader.invoke({"question":query, "document": doc_txt})
        print(f"Anthropic Document {idx + 1} relevance score: {binary_score}")
        relevance_scores[f"Anthropic_Doc_{idx+1}"] = binary_score
        relevance_docs[f"Anthropic_Doc_{idx+1}"] = doc_txt

    for idx, doc in enumerate(docs_keivan):
        doc_txt = doc.page_content
        binary_score = retrieval_grader.invoke({"question": query, "document": doc_txt})
        print(f"Keivan Document {idx + 1} relevance score: {binary_score}")
        relevance_scores[f"Keivan_Doc_{idx + 1}"] = binary_score
        relevance_docs[f"Keivan_Doc_{idx + 1}"] = doc_txt

    return relevance_scores, relevance_docs

In [22]:
data_score = assess_retrieved_docs("What is Rag??")

Anthropic Document 1 relevance score: no
Anthropic Document 2 relevance score: yes
Anthropic Document 3 relevance score: no
Anthropic Document 4 relevance score: no
Keivan Document 1 relevance score: no
Keivan Document 2 relevance score: no
Keivan Document 3 relevance score: no
Keivan Document 4 relevance score: no


In [23]:
data_score

({'Anthropic_Doc_1': 'no',
  'Anthropic_Doc_2': 'yes',
  'Anthropic_Doc_3': 'no',
  'Anthropic_Doc_4': 'no',
  'Keivan_Doc_1': 'no',
  'Keivan_Doc_2': 'no',
  'Keivan_Doc_3': 'no',
  'Keivan_Doc_4': 'no'},
 {'Anthropic_Doc_1': 'safety-trained response from the model is still triggered — the LLM will likely respond that it can’t help with the request, because it appears to involve dangerous and/or illegal activity.However, simply including a very large number of faux dialogues preceding the final question—in our research, we tested up to 256—produces a very different response. As illustrated in the stylized figure below, a large number of “shots” (each shot being one faux dialogue) jailbreaks the model, and causes it to provide an answer to the final, potentially-dangerous request, overriding its safety training.Many-shot jailbreaking is a simple long-context attack that uses a large number of demonstrations to steer model behavior. Note that each “...” stands in for a full answer to th

In [17]:
from langchain_core.output_parsers import StrOutputParser

def rewrite_query(query):
    """Rewrite and optimize a given user query for the model."""
    question_rewriter = (re_write_prompt
                         | llm_model_groq
                         | StrOutputParser())
    
    return question_rewriter.invoke({"question":query})

In [18]:
rewrite_query("Who is Keivan?")

'Who is Keivan and what is he known for?'

In [19]:
from langchain.schema import Document
def search_web(query):
    """Search the web  for complimentary information."""
    docs = web_search_tool.invoke({"query":query})
    web_docs = "\n".join([doc["content"] for doc in docs])
    return Document(page_content=web_docs)

In [29]:
def generate_answer(docs, query):
    rag_chain = (prompt
                 | llm_model_groq
                 | StrOutputParser())
    
    return rag_chain.invoke({"context": docs, "question":query})

In [31]:
def main(query):
    binary_score, binary_docs = assess_retrieved_docs(query)
    docs = []
    for doc, score in binary_score.items():
        if score == "yes":
            docs.append(binary_docs[doc])
    
    if not docs:
        print("[INFO] Searching the web.")
        docs = search_web(query)

    optimized_query = rewrite_query(query)

    return generate_answer(docs, optimized_query)
    


In [33]:
q = "What is the of Tom cruises mother??"
display(Markdown(main(q)))


Anthropic Document 1 relevance score: no
Anthropic Document 2 relevance score: no
Anthropic Document 3 relevance score: no
Anthropic Document 4 relevance score: no
Keivan Document 1 relevance score: no
Keivan Document 2 relevance score: no
Keivan Document 3 relevance score: no
Keivan Document 4 relevance score: no
[INFO] Searching the web.


Tom Cruise's mother was Mary Lee South, born Mary Lee Pfeiffer. She was a special-education teacher and later married Jack South in 1978. She passed away at the age of 80.